<a href="https://colab.research.google.com/github/GiackAloZ/di-playstore/blob/master/play-store.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Progetto di DI - Google Play Store: Category Classification

Questo progetto prende in esame un dataset ottenuto dal Google Play Store, dove sono state collezionate le informazioni riguardanti piu' di 10.000 app.

Lo scopo del progetto e' quello di utilizzare semplici tecniche di NLP per classificare una app nella sua categoria (eg. Game, Social, Art & Design, etc...) a partire dal suo nome.

Ho trovato il dataset su Kaggle ed e' consultabile [qui](https://www.kaggle.com/lava18/google-play-store-apps).



## Descrizione del problema e analisi esplorativa

Si deve realizzare un modello che, dato il nome di una app, la classifichi in base alla sua categoria tra le varie disponibili (eg. Game, Social, Art & Design, etc...)

Per prima cosa, importiamo le librerie che ci serviranno.

In [ ]:
import os

import numpy as np
import pandas as pd
import sklearn as skl
import tensorflow.keras as ks
import wordcloud
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
%matplotlib inline

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, StratifiedKFold, StratifiedShuffleSplit
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Perceptron, LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import precision_score, recall_score, f1_score

from sklearn.base import BaseEstimator, ClassifierMixin

from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

nltk.download("punkt")
nltk.download('averaged_perceptron_tagger')
nltk.download("stopwords")
nltk.download("wordnet")

### Caricamento e pulizia del dataset

Carichiamo i dati dal csv `googleplaystore.csv` sulla repo GitHub come un dataframe pandas e diamo un'occhiata alla sua shape e alle prime 5 righe.

In [ ]:
apps_data = pd.read_csv("https://raw.githubusercontent.com/GiackAloZ/di-playstore/master/data/googleplaystore.csv")
print(apps_data.shape)
apps_data.head(5)

Ci sono numerose informazioni in questo dataset, ma le colonne che andremo a considerare sono due:

- `App`, ovvero i nomi delle app
- `Category`, la categoria a cui appartiene ogni app

Andiamo a osservare quali possono essere le categorie a cui un'applicazione puo' appartenere.

In [ ]:
apps_data["Category"].value_counts()

C'e' una categoria che probabilemente e' stata inserita per errore, cioe' la categoria identificata con il nome "1.9", perche' ha una sola occorrenza. Andiamo a vedere di cosa si tratta.

In [ ]:
wrong_data = apps_data[apps_data["Category"] == "1.9"]
wrong_data

Sembrerebbe un dato errato, infatti ha tutte le colonne sbagliate o shiftate. Possiamo rimuoverla dal dataset.

In [ ]:
apps_data = apps_data[~apps_data.index.isin(wrong_data.index)].reset_index()

### Analisi esplorativa delle feature utilizzate

Andiamo ora a visualizzare le categorie in un grafico a barre e in un grafico a torta.

In [ ]:
apps_data["Category"].value_counts().plot.bar()

In [ ]:
plt.figure(figsize=(10,10))
apps_data["Category"].value_counts().plot.pie()

Notiamo che le categorie spaziano da molto popolate (eg. FAMILY o GAME) a scarsamente popolate (eg. COMICS o BEAUTY).
Questo potrebbe causare problemi di sbilanciamento di classi.

Vogliamo ora farci un'idea di come sono fatti i nomi delle app. Per fare cio', andiamo a concatenare tutti i nomi in una string `text` per poi creare una `cloudword` di tutti i nomi delle app in modo da osservare a colpo d'occhio quali sono le parole che compaiono piu' spesso.

In [ ]:
text = " ".join(apps_data["App"])

In [ ]:
wc = wordcloud.WordCloud(
    width=800, height=800,
    background_color="white", max_words=200
).generate(text)

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(wc)

Notiamo subito che le parole piu' utilizzate sono quelle che ci potevamo aspettare, come "App" o "Mobile" oppure anche "Free".

### Correlazione tra categoria di appartenenza e nomi delle app

Andiamo ora a ottenere le top 5 parole per ogni categoria.
Per farlo, raggruppiamo il dataset per categoria e concateniamo i nomi delle app.

In [ ]:
names_category = apps_data.groupby("Category")["App"].agg(" ".join)
names_category.head(5)

Poi andiamo a contare le frequenze delle parole di ogni categoria in una BagOfWords utilizzando `CountVectorizer`

In [ ]:
vect = CountVectorizer()
bow = vect.fit_transform(names_category)
bow.shape

Ok, il `CountVectorizer` ha rilevato 8714 diverse parole. Ora andiamo a ordinarle per frequenza definendo una funzione che ci restituisce le k parole piu' frequenti in una categoria.

In [ ]:
def most_k_freq(cat_freqs, vect, k=1):
    word_freqs = [(word, cat_freqs[0, i]) for word, i in vect.vocabulary_.items()]
    word_freqs_sorted = sorted(word_freqs, key=lambda x: x[1], reverse=True)
    return word_freqs_sorted[:k]

Quindi per ogni riga della BagOfWords andiamo ad estrarre le k parole piu' frequenti.
Controlliamo stampando la parole piu' frequenti della prima categoria.
Ogni elemento della lista `top_words` e' una tupla con (`word`, `frequency`).


In [ ]:
k = 5
top_words = [most_k_freq(row, vect, k) for row in bow]
print(top_words[0])

Mettiamo il risultato in un dataframe, usando un `MultiIndex` per suddividerlo meglio.
Per farlo, dobbiamo prima _flattare_ la lista di liste di tuple `top_words` per renderla una lista di liste.

In [ ]:
top_words_flatten = [[x for y in cat for x in y] for cat in top_words]

ranking_labels = np.arange(k) + 1
word_labels = ["word", "freq"]

top_words_df = pd.DataFrame(
    top_words_flatten, 
    index=names_category.index,
    columns=pd.MultiIndex.from_product(
        [ranking_labels, word_labels], names=["ranking", "word_freq"]
    )
)
top_words_df.head(5)

Andiamo a guardare le statistiche aggregate del dataframe appena creato.

In [ ]:
top_words_df.describe()

Vediamo come la media della parola piu' frequente per ogni categoria e' di circa 50, mentre gia' alla 5a la frequenza media e' di circa 20.

Questo nota il fatto che i nomi delle app della stessa categoria sono piu' o meno simili tra loro, o comunque hanno dei termini ricorrenti.
Quindi, abbiamo una certa correlazione tra categoria della app e nome di essa.

## Preprocessing, tokenizzazione ed estrazione delle features dai nomi delle app

Uno dei passi fondamentali in un problema di NLP e' l'estrazione delle features.

TODO

In [ ]:
vect_no_tok = TfidfVectorizer()
X_no_tok = vect_no_tok.fit_transform(apps_data["App"])
print(f"Number of tokens : {len(vect_no_tok.get_feature_names())}")

Usando il tokenizer di default, abbiamo 8714 token differenti.

Possiamo ridurre un po' il numero di token anche senza cambiare il tokenizer, ma solo togliendo quei token che compaiono meno di 3 volte nei nomi delle app.



In [ ]:
vect_no_tok = TfidfVectorizer(min_df=3)
X_no_tok = vect_no_tok.fit_transform(apps_data["App"])
print(f"Number of tokens : {len(vect_no_tok.get_feature_names())}")

Gia' togliendo i token con frequenza minore di 3 riduciamo abbondantemente il numero di token da 8714 a 2336 quindi di oltre 3 volte e mezzo.

Andiamo ora a provare lo stemming e la lemmatizzazione. Nelle due funzioni filtriamo anche le stepwords considerate tali da `nltk` e consideriamo solo i token che sono alfabetici e che hanno lunghezza maggiore di 1.

In [ ]:
def get_tokens_no_stopwords_alpha(texts):
    tokens = nltk.tokenize.word_tokenize(texts)
    #token filtering (not stopword and alphabetic)
    return {token for token in tokens
                  if token not in nltk.corpus.stopwords.words("english")
                     and token.isalpha()}

In [ ]:
def tokenizer_stem(app_names):
    tokens = get_tokens_no_stopwords_alpha(app_names)
    #stemming
    stemmer = nltk.stem.PorterStemmer()
    stemmed_tokens = {stemmer.stem(token) for token in tokens}
    #one-char token removal
    return [token for token in stemmed_tokens if len(token) > 1]

In [ ]:
vect_stem = TfidfVectorizer(min_df=3, tokenizer=tokenizer_stem)
X_stem = vect_stem.fit_transform(apps_data["App"])
print(f"Number of tokens : {len(vect_stem.get_feature_names())}")

In [ ]:
def tokenizer_lemm(app_names):
    tokens = get_tokens_no_stopwords_alpha(app_names)
    #lemmatizzazione
    lemmatizer = nltk.wordnet.WordNetLemmatizer()
    lemmatized_tokens = {lemmatizer.lemmatize(token) for token in tokens}
    #one-char token removal
    return [token for token in lemmatized_tokens]

In [ ]:
vect_lemm = TfidfVectorizer(min_df=3, tokenizer=tokenizer_lemm)
X_lemm = vect_lemm.fit_transform(apps_data["App"])
print(f"Number of tokens : {len(vect_lemm.get_feature_names())}")

Non c'e' una grossa differenza usando stemming o lemmatizzazione. Usiamo la lemmatizzazione siccome e', in genere, piu' precisa.

Intanto, andiamo a vedere la lunghezza media dei token dopo la lemmatizzazione.

In [ ]:
tokens, indexes = zip(*vect_lemm.vocabulary_.items())
tokens_df = pd.DataFrame(tokens, columns=["token"], index=indexes)
tokens_df["length"] = tokens_df["token"].map(len)
tokens_df["length"].describe()

E' poco superiore a 5 e mezzo, quindi indica che anche se abbiamo usato il lemming, non abbiamo tolto troppo contenuto dalle parole. 

## Generazione di diversi modelli di learning

TODO

### Divisione train e test set

In [ ]:
X = apps_data["App"]
y = apps_data["Category"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=1/6,
    stratify=y
)

### Scelta feature

In [ ]:
vect_lemm = TfidfVectorizer(tokenizer=tokenizer_lemm)
X_train_lemm = vect_lemm.fit_transform(X_train)
X_test_lemm = vect_lemm.transform(X_test)
len(vect_lemm.get_feature_names())

In [ ]:
(X_train_lemm.sum(axis=1) == 0).sum()

### Perceptron

In [ ]:
param_perceptron = {
    "penalty": ["l1", "l2"],
    "alpha": np.logspace(-5, -2, num=4)
}

search_perceptron = GridSearchCV(
    Perceptron(random_state=42),
    param_grid=param_perceptron,
    cv=StratifiedKFold(n_splits=5),
    verbose=2
)
search_perceptron.fit(X_train_lemm, y_train)

In [ ]:
search_perceptron.score(X_test_lemm, y_test)

In [ ]:
search_perceptron.best_params_

### Logistic regression

In [ ]:
param_logistic = [
    {
        "penalty": ["l1", "l2"],
        "C": np.logspace(0, 1, num=2)
    },
    {
        "penalty": ["elasticnet"],
        "C": np.logspace(0, 1, num=2),
        "l1_ratio": [0.1, 0.5]
    }
]

search_logistic = GridSearchCV(
    LogisticRegression(random_state=42, solver="saga"),
    param_grid=param_logistic,
    cv=StratifiedKFold(n_splits=5),
    verbose=2
)
search_logistic.fit(X_train_lemm, y_train)

In [ ]:
search_logistic.score(X_test_lemm, y_test)

In [ ]:
search_logistic.best_params_

### SVC

In [ ]:
param_svc = {
    "C" : np.logspace(0, 2, num=3),
    "loss" : ["hinge", "squared_hinge"]
}

search_svc = GridSearchCV(
    LinearSVC(random_state=42, penalty="l2"),
    param_grid=param_svc,
    cv=StratifiedKFold(n_splits=5),
    verbose=2
)
search_svc.fit(X_train_lemm, y_train)

In [ ]:
search_svc.score(X_test_lemm, y_test)

In [ ]:
search_svc.best_params_

### Multi-layer perceptron

In [ ]:
param_mlp = {
    "hidden_layer_sizes" : [(size,) for size in np.logspace(4, 6, num=3, base=2, dtype=np.int)],
    "alpha" : np.logspace(-5, -3, num=3)
}

search_mlp = GridSearchCV(
    MLPClassifier(random_state=42),
    param_grid=param_mlp,
    cv=StratifiedKFold(n_splits=5),
    verbose=2
)
search_mlp.fit(X_train_lemm, y_train)

In [ ]:
search_mlp.score(X_test_lemm, y_test)

In [ ]:
search_mlp.best_params_

### Keras neural network

In [ ]:
label_encoder = LabelEncoder()

y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

In [ ]:
def keras_model(hidden_layer_sizes=(64,64), reg_rate=1e-4):
    model = ks.models.Sequential([
        ks.Input(shape=(X_train_lemm.shape[1],))] + [
        ks.layers.Dense(size,
                        activation="relu",
                        kernel_regularizer=ks.regularizers.l2(reg_rate)) for size in hidden_layer_sizes
    ] + [ks.layers.Dense(y.unique().size, activation="softmax")]
    )
    
    model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["acc"])
    
    return model

keras_model().summary()

In [ ]:
class KerasModel(BaseEstimator, ClassifierMixin):
    def __init__(self,
                 hidden_layer_sizes=(64,64),
                 reg_rate=1e-4):
        # label encoder (string -> int)
        self._encoder = LabelEncoder()
        self.hidden_layer_sizes = hidden_layer_sizes
        self.reg_rate = reg_rate
    
    def fit(self, X, y, **kargs):
        # encode string classes to integers
        y_encoded = self._encoder.fit_transform(y)

        # keras model with multiple hidden layers with "relu" activation
        # and one final layer with "softmax" activation for mutli-class classification
        self._model = ks.models.Sequential([
            ks.Input(shape=X.shape[1:2])] + [     # input layer with shape equal to number of classes 
            ks.layers.Dense(size,
                            activation="relu",
                            kernel_regularizer=ks.regularizers.l2(reg_rate)) for size in hidden_layer_sizes
            ] + [ks.layers.Dense(self._encoder.classes_.size, activation="softmax")]
        )
        
        self._model.compile(
            optimizer="adam",
            loss="sparse_categorical_crossentropy",
            metrics=["acc"]
        )

        # show only epoch number while fitting
        self._model.fit(X, y_encoded, verbose=10, **kargs)
        return self
    
    def predict(self, X):
        y_pred = self._model.predict_classes(X)
        # inverse transform classes (int -> string)
        return self._encoder.inverse_transform(y_pred)

In [ ]:
[tuple(size//div for div in np.logspace(0, 2, num=3, base=2, dtype=np.int))
                 for size in np.logspace(7, 9, num=3, base=2, dtype=np.int)]

In [ ]:
param_keras = {
    "hidden_layer_sizes": [
        tuple(size//div for div in np.logspace(0, 2, num=3, base=2, dtype=np.int))
                        for size in np.logspace(8, 9, num=2, base=2, dtype=np.int)],
    "reg_rate": np.logspace(-4, -3, num=3)
}

search_keras = GridSearchCV(
    KerasModel(),
    param_grid=param_keras,
    cv=StratifiedKFold(n_splits=5),
    verbose=2
)
search_keras.fit(X_train_lemm.toarray(), y_train, batch_size=256, epochs=50)

In [ ]:
search_keras.score(X_test_lemm.toarray(), y_test)

## Model evaluation

In [ ]:
def confidence(acc, N, Z):
    den = (2*(N+Z**2))
    var = (Z*np.sqrt(Z**2+4*N*acc-4*N*acc**2)) / den
    a = (2*N*acc+Z**2) / den
    inf = a - var
    sup = a + var
    return (inf, sup)

In [ ]:
confidence(0.5124515771997786, len(X_test), 1.96)

In [ ]:
def compare_confidence(acc1, acc2, N, Z):
    var_sq = acc1 * (1 - acc1) / N + acc2 * (1 - acc2) / N
    a = abs(acc1 - acc2)
    inf = a - Z * np.sqrt(var_sq)
    sup = a + Z * np.sqrt(var_sq)
    return (inf, sup)

In [ ]:
compare_confidence(0.12, 0.5124515771997786, len(X_test), 2.56)

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()